<a href="https://colab.research.google.com/github/ccc-frankfurt/Practical_ML_SS21/blob/main/Naive_Bayes_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Naive Bayes and SMS Spam Collection Dataset

In this week we will use Naive Bayes (the assumption of feature independence in a probabilistic context) in order to accomplish a classification task of distinguishing spam from non-spam messages.

We will use the kaggle SMS Spam Collection dataset for this purpose: https://www.kaggle.com/uciml/sms-spam-collection-dataset

First, let us import the packages that we will need. Notice that we are importing nltk - which is a Natural Language Toolkit which will aid us in preprocessing the dataset.

In [ ]:
# please restart the runtime after running this cell (to use the versioned matplotlib)
!pip install matplotlib==3.1.0

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import string

from sklearn.metrics import confusion_matrix

import nltk
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

!pip install wget
import wget

In [ ]:
# download the data
url = "https://raw.githubusercontent.com/mohitgupta-omg/Kaggle-SMS-Spam-Collection-Dataset-/master/spam.csv"
data = wget.download(url)

We will convert the data into the pandas dataframe and remove the columns we do not need. We will add an additional column with labels, converted to integers (instead of strings 'spam' and 'ham').

In [ ]:
df = pd.read_csv('./spam.csv', encoding = 'latin-1')
# delete columns with unuseful values
df = df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1)
df.columns = ['label', 'text']

# convert string labels to integers
df["label_int"] = //TODO
df.head(n=10)

As a preprocessing step, we have to clean the dataset. This is because not all sms-symbols are relevant for spam classification. For example, punctuation is not. Consider also that word endings, e.g. singular or plural, might hinder to recognize the word as the same (and update the count accordingly).

In [ ]:
# the cleanTest function is from 
# https://www.kaggle.com/ishansoni/sms-spam-collection-dataset

from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("english")

def cleanText(message):
    
    # remove the punctuation from the messages
    # hint1: use maketrans method of str
    # hint2: you can get punctuation characters with string.punctuation (print those to check)
    message = message.translate(//TODO)
    # convert words in the message into their stems and remove stopwords
    # also remove numbers hint: isdigit() method could be useful
    words = [//TODO]
    
    return " ".join(words)

df["text"] = df["text"].apply(cleanText)
df.head(n = 10)    

Split the data into train and test.

In [ ]:
x = df["text"]
y = df["label_int"]

perm = np.random.permutation(len(x))

split = 0.8

x_train, x_test = x[perm[:int(split*len(x))]], x[perm[int(split*len(x)):]]
y_train, y_test = y[perm[:int(split*len(y))]], y[perm[int(split*len(y)):]]

Gather all spam words in a list and all ham words in another list. 

Hint: a message consists of words. Join the messages into a global string and then split them.

In [ ]:
# 0 - ham, 1 - spam
# select all train spam messages
spam_messages = //TODO
# join them and split into a list of single words
spam_words = //TODO
print(spam_words)

# repeat for ham messages
ham_messages = //TODO
ham_words = //TODO
print(ham_words)

In [ ]:
# calculate the prior of a message being spam or non-spam

ham_prob = //TODO
spam_prob = //TODO

print('probability of a message being ham: ',ham_prob)
print('probability of a message being spam: ',spam_prob)

Calculate the frequencies (counts) for all words given a category (spam or ham) and also one of the normalizing factors (number of word occurences in the category, meaning the sum of the frequencies of all words in a category).

In [ ]:
def get_word_counts_for_class(words):
    word_df = pd.DataFrame({'words':words})

    # calculate the frequencies for each word in the data frame
    word_counts = //TODO 

    # add up all word counts (frequencies)
    words_num = word_counts.to_numpy().sum()
    print('number of words: '+str(word_counts.count()))
    print('number of word occurences: ' + str(words_num)+ '\n')

    # for better interpretability rename columns
    word_counts = word_counts.reset_index()
    word_counts.columns = ['words','counts']

    print('words with the heighest count:')
    print(word_counts[:10])
    print('>----------------------<')

    # return the counts (frequencies) of words, as well as the sum of all frequencies
    return word_counts, float(words_num)

In [ ]:
# calculate the frequency of words in the spam and non-spam categories
# use the function implemented above

# Attention: we already name the variable probs, 
# but the normalization of counts is yet to be done
spam_word_probs, spam_count_sum = get_word_counts_for_class(spam_words)
ham_word_probs, ham_count_sum = get_word_counts_for_class(ham_words)

# concatenate the spam and non-spam words
# and calculate the number of words in the vocabulary
spam_words_selected = spam_word_probs['words'].to_numpy()
ham_words_selected = ham_word_probs['words'].to_numpy()
vocabulary = //TODO
num_words_voc = len(vocabulary)

print('number of words in the vocabulary')
print(num_words_voc)

In [ ]:
# calculate word probabilities for categories spam and non-spam:
# Laplace smoothing for naive Bayes (normalize the counts)
spam_word_probs['probs'] = //TODO
print(spam_word_probs.head())

ham_word_probs['probs'] = //TODO
print(ham_word_probs.head())

In [ ]:
# write a function to get a log-probability of a chosen word belonging to the category
def get_logprob(word, frame, word_count, voc_size, otherframe):

    # if the word was encountered in the category:
    row =  frame.loc[frame['words'] == word]
    if not(row.empty):
        return //TODO
    
    # if the word was not encountered (count=0) in a category, 
    # but is part of the vocabulary
    row = otherframe.loc[otherframe['words'] == word]
    if not(row.empty):
        return np.log(1./(word_count+voc_size))

    return 0

# test for the word 'call' the probability of a message being spam or not
print(get_logprob('call',spam_word_probs, spam_count_sum, num_words_voc, ham_word_probs))
print(get_logprob('call',ham_word_probs, ham_count_sum, num_words_voc, spam_word_probs))

In [ ]:
# test message for spam using Naive Bayes approach
def predict_message_labels(message):
    # calculate the probability of a message being spam
    words = message.split(' ')

    # get the probabilities of single words
    spam_word_logprobs_m = list(map(lambda x: 
                                    get_logprob(x,spam_word_probs, 
                                                spam_count_sum, num_words_voc, 
                                                ham_word_probs),words))
    # calculate the message probability given word probabilities
    spam_logprob_m = //TODO
    
    # calculate the probability of a message being ham
    ham_word_logprobs_m = list(map(lambda x: 
                                   get_logprob(x,ham_word_probs,
                                               ham_count_sum, num_words_voc, 
                                               spam_word_probs),words))
    ham_logprob_m = //TODO
   
    # decide
    //TODO

In [ ]:
# pick a random text example, print the text and label and check the prediction

example = df['text'][22]
print(example)
print('true label: ',df['label_int'][22])
print('predicted label: ',predict_message_labels(example))

In [ ]:
# apply the prediction method to all messages in the x_train and check the accuracy
train_preds = x_train.apply(predict_message_labels)
print('accuracy for the trainset:')
((train_preds==y_train).sum())/len(y_train)

Visualize your results: check how many messages have been classified correctly, or misclassified.

In [ ]:
def visualize_confusion_matrix(true_vals, pred_vals):
    cm = confusion_matrix(true_vals, pred_vals)
    index = ['predicted ham', 'predicted spam']  
    columns = ['ham', 'spam']  
    cm_df = pd.DataFrame(cm,columns,index)                      
    fig, ax = plt.subplots(figsize=(10,6))  
    sns.heatmap(cm_df,annot=True,cmap='Blues', fmt='g')

In [ ]:
visualize_confusion_matrix(y_train, train_preds)

Do the same for your test set on which you have not trained to actually validate the results.

In [ ]:
test_preds = x_test.apply(predict_message_labels)
test_acc = ((test_preds==y_test).sum())/len(y_test)
print('accuracy for the testset: ',test_acc)

visualize_confusion_matrix(y_test, test_preds)